<h1>Pose Detection Implemented with mediaPipe and Opencv<h1>

In [1]:
#Load Libraries
import cv2
import mediapipe as mp
import os
import sys
import numpy as np
import csv

In [2]:
#Declare  Some Variables

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

ROOTPATH  =  working_dir =os.path.join(os.getcwd(),'Poses') #Path to the folder with images  Poses folder contains images 
IMAGEEXTENTION = ('.jpg', '.png', 'jpeg','JPG','.webp') #Declaare valid image extensions
WIDTH = 900 #Set width value
HEIGHT = 600 #Set height value

In [3]:
#Function for Body Landmarks Detection.  
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

In [4]:
#image_files is dictionary data structure, key = pose title  value = pose image storage address
#Full_List is dictionary data structure contains output results 
def ExtractPointsandSaveinCSV(image_files, Full_List):
    
    for keys, b in image_files.items():
        
        with mp_holistic.Holistic(
            static_image_mode=True,
            model_complexity=2,
            enable_segmentation=True,
            refine_face_landmarks=True) as holistic:
            
            image = cv2.imread(b)
            image = cv2.resize(image, (WIDTH,HEIGHT), interpolation = cv2.INTER_LINEAR)
    
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if results.pose_landmarks:
                
                a = {11 : 11, 12 : 12,
                     13 : 13, 14 : 14,
                     15 : 15, 16 : 16,
                     23 : 23, 24 : 24,
                     25 : 25, 26 : 26,
                     27 : 27, 28 : 28}
                
                for i in (a.keys()):
                    key = a[i]
                    cordinates = []
                    X = int(np.round(results.pose_landmarks.landmark[key].x * WIDTH))
                    Y = int(np.round(results.pose_landmarks.landmark[key].y * HEIGHT))
                    cordinates.append(X)
                    cordinates.append(Y)
                    a[i] = cordinates
                Full_List[keys] = a


In [5]:
#Draw body landmarks 

def restore_landmarks(pose_map, image, color, thickness, color_circle = (255,0,0), radious = 10, text = ' '):
    
    for indx, pose in enumerate(pose_map.keys()):
        cv2.circle(image, pose_map[pose], radious, color_circle, -thickness)
        if pose == 11:
            cv2.line(image, pose_map[pose], pose_map[13], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[23], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[12], color, thickness)
        elif pose == 12:
            cv2.line(image, pose_map[pose], pose_map[14], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[24], color, thickness)
        elif pose == 13:
            cv2.line(image, pose_map[pose], pose_map[15], color, thickness)
        elif pose == 14:
            cv2.line(image, pose_map[pose], pose_map[16], color, thickness)
        elif pose == 23:
            cv2.line(image, pose_map[pose], pose_map[25], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[24], color, thickness)
        elif pose == 24:
            cv2.line(image, pose_map[pose], pose_map[26], color, thickness)
        elif pose == 25:
            cv2.line(image, pose_map[pose], pose_map[27], color, thickness)
        elif pose == 26:
            cv2.line(image, pose_map[pose], pose_map[28], color, thickness)
        position = (10,50)
        cv2.putText(image,text,position, cv2.FONT_HERSHEY_SIMPLEX, 2,(209, 80, 0, 255),2)
        

In [6]:
#Calculate accurance of user Pose and Standart Pose 
def calculate_accurancy(standart_pose, user_pose):
    accurancy = {}
    standart_list = list(standart_pose.items())
    user_list = list(user_pose.items())
    for i in range(len(standart_pose)):
        point=standart_list[i][0]
        standart = standart_list[i][1]
        value = user_list[i][1]
        absolute = [abs(standart[0]-value[0]),abs(standart[1]-value[1])]
        try:
            a = absolute[0]/value[0]
            b = absolute[1]/value[1]
        except ZeroDivisionError:
            a = value[0]
            b = value[1]
        relative = [round((a*100),1),round((b*100),1)]
        accurancy[point]=relative
    return accurancy

In [7]:
def save_landmarks(results):
    a= results.pose_landmarks
    landmark_map = {11 : 11, 12 : 12,
                    13 : 13, 14 : 14,
                    15 : 15, 16 : 16,
                    23 : 23, 24 : 24,
                    25 : 25, 26 : 26,
                    27 : 27, 28 : 28}
    

    for i in (landmark_map.keys()):
        key = landmark_map[i]
        cordinates = []
        X = int(np.round(results.pose_landmarks.landmark[key].x * WIDTH))
        Y = int(np.round(results.pose_landmarks.landmark[key].y * HEIGHT))
        cordinates.append(X)
        cordinates.append(Y)
        landmark_map[i] = cordinates
    return landmark_map

In [8]:
def getImageList(poseList):
    for indx, file_name in enumerate(os.listdir(ROOTPATH)):
        if file_name.endswith(IMAGEEXTENTION):
            file_path=os.path.join(ROOTPATH, file_name)
            poseList[file_name[0:(file_name.rfind('.'))]]=file_path
   

<h1>DEMONSTRATION</h1>

In [9]:
poseList= {}
imageMarks = {}
getImageList(poseList)
ExtractPointsandSaveinCSV(poseList, imageMarks)

In [10]:
#Example with static images 
for key, val in imageMarks.items():
    Pose = imageMarks[key]
    
    frame = np.zeros((HEIGHT, WIDTH, 3), dtype = "uint8")
    restore_landmarks(Pose, frame, (255,0,0), 1, (0,255,0), 5, str(key)) #RGB(255,0,0) line color RGB(0,255,0) circle color 
    cv2.imshow((str(key)), frame) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()
          

In [11]:
#Example with videostream
for key, val in imageMarks.items():
    Pose = imageMarks[key]
    vid = cv2.VideoCapture(0)
    while(True):
        ret, frame = vid.read()
        frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation= cv2.INTER_LINEAR)
        frame = cv2.flip(frame,1)
        restore_landmarks(Pose,frame,(255,0,0),5)
        cv2.imshow(str(key), frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): #press q to exit loop
            break
    vid.release()
    cv2.destroyAllWindows()   

In [12]:
#Real time pose detection with calculations 
#If match line green 
Pose = imageMarks['WarriorPose']


for key, val in imageMarks.items():
    cap = cv2.VideoCapture(0)
    Pose = imageMarks[key]
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue
          
            image = cv2.resize(image, (WIDTH,HEIGHT), interpolation= cv2.INTER_LINEAR)
            image = cv2.flip(image,1)
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                restore_landmarks(Pose,image,(255,0,0),10)
                user_pose = save_landmarks(results)
                results = calculate_accurancy(Pose, user_pose)
                results_list = list(results.items())
                for i in range(len(results)):
                    a = results_list[i][1]
                    if a[0] > 0.5 and a[1] > 0.5:
                        print("Not Matched")
                        break
                    else:
                        restore_landmarks(Pose,image,(0,255,0),10, (0,255,0), 10, text = 'Matched')
                        print("Matched")
            cv2.imshow('MediaPipe Holistic', image)
            if cv2.waitKey(5) & 0xFF == 27: #Press Esc to exit 
                break
        cap.release()
        cv2.destroyAllWindows()

Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not 